In [ ]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime, openpyxl
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup 
import nltk
from nltk.corpus import stopwords 
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

#df, code = generate_code_from_db(ctx, qgroup=4028)

In [71]:
f=r'C:\Users\timko\Downloads\CyberScope PQC Template - Updated - 020823.xlsx'
usecols='A:X,AJ,CZ:DI'
df=pd.read_excel(f'{f}', sheet_name=0, header=0, usecols='Y:AF',index_col=0).reset_index()
for c in df.columns:
    df.rename({c: re.sub('','',c.strip())}, axis=1, inplace=True)
    if 'Unnamed' in c: df.drop(columns=c, inplace=True)
#df.columns


In [72]:
df.columns

Index(['CryptoAlgorithmUsed', 'LengthCryptoKeysModule', 'HostingInfo',
       'CloudServiceProviders', 'GovernmentCloudProviders',
       'HybridCloudProviders', 'FIPS199SystemCategorization',
       'SoftwarePackageType'],
      dtype='object')

In [34]:
ldf=[]
for c in df.columns:
    t=df.loc[0][c] 
    icn=c.strip()
    c=re.sub('[^A-Za-z0-9]','',c.strip())
    c=re.sub('graphic|uirement|whichincludes|imateof|rmation|eloper|eralWorkforceK|estmentsK|tractorsK|includingMajorandMinorversion|COTSGOTSNDIOther','',c ) 
    c=c.replace('ofassociatedcryptokeysormodules','CryptoKeysModules') 
    sql=f'[{c}] INT NULL,'
    vb=f'BindDDL(e, "{c}", "ddl{c}", "{c}")'
 
    if type(t)!=str: 
        sql=f'[{c}] [NVARCHAR](4000) NULL,' 
        vb=f'DirectCast(e.Item.FindControl($"{c}"), RadTextBox).Text = drVal("{c}").ToString()'
    ldf.append({'col':c, 'icn': icn, 'sql':sql, 'vb':vb })


In [58]:
t = """
other Has the as Languages technologies platforms is your web don't we have web driver 
"""
def encode(t):
    if re.search('.*OTHER\s.*',str(t).strip().upper()):
        return 'OTH'
    t = ' '.join([w[:15] for w in str(t).split(' ') if w not in sw]).upper().strip()   
    t = re.sub('[^A-Z0-9]', '', t ) 
    if len(t) > 10:
        t = re.sub('[AEIOU]', '', t)   
    if len(t) > 10:
        t=f'{t[:8]}{t[len(t)-2:len(t)]}'  
    return t[:10]
encode(t)

'OTH'

In [67]:

def generate_picklist(data, col, PK_PicklistType=500, PK_PickList=5000):
    with open(ctx.get_tempalte_dir()+'picklists.sql', 'r') as f:
        pl=f.read() 
    with open(ctx.get_tempalte_dir()+'plt.sql', 'r') as f:
        plt='\n'+f.read() 
    dff=data  
    dff.applymap(lambda x: re.sub('[^\x00-\x7F]','',str(x)))
    dff=dff.loc[:,col] 
    dff.dropna(inplace=True) 
    rslt = ''
    for i,s in enumerate(list(dff)):
        rslt=rslt+','+pl.replace('{DisplayValue}',str(s)).replace('{CodeValue}',encode(s)).replace('{SORT}',str(i+1)).replace('{PK_PickList}',str(PK_PickList+i))
    rslt=rslt[1:].replace(',(','\n,(')  
    plt=plt.replace('{picklists.sql}',rslt)
    plt=plt.replace('{Description}',col)
    plt=plt.replace('{UsageField}',col)
    plt=plt.replace('{PK_PickListType}',str(PK_PicklistType))
    return plt, i
txt=''
k=5000
for i,c in enumerate(df.columns):
    k=k+10+gp[1]
    gp=generate_picklist(df,c,i+475, i+k)
    txt=txt+gp[0] 
with open(f'{ctx.get_dest()}script.sql', 'w', encoding='UTF-8') as fw:
    fw.write(txt)

In [ ]:
with open(ctx.get_tempalte_dir()+'picklists.sql', 'r') as f:
    pl=f.read() 
with open(ctx.get_tempalte_dir()+'plt.sql', 'r') as f:
    plt=f.read()  
 


In [36]:

with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
    t=f.read() 
    for d in ldf: 
        print(t.replace("{col}",d['col']).replace("{ht}",d['icn'])  ) 

<telerik:GridTemplateColumn HeaderText="Agency Name" UniqueName="AgencyName" SortExpression="AgencyName">
<EditItemTemplate>
    <telerik:RadTextBox ID="AgencyName" runat="server" ></telerik:RadTextBox>
<%-- <telerik:RadDropDownList ID="AgencyName" runat="server"></telerik:RadDropDownList> --%>
</EditItemTemplate>
<ItemTemplate>
    <asp:Label ID="lblAgencyName" runat="server" Text='<%# Bind("AgencyName") %>'></asp:Label>
</ItemTemplate>
</telerik:GridTemplateColumn>
<telerik:GridTemplateColumn HeaderText="System Name" UniqueName="SystemName" SortExpression="SystemName">
<EditItemTemplate>
    <telerik:RadTextBox ID="SystemName" runat="server" ></telerik:RadTextBox>
<%-- <telerik:RadDropDownList ID="SystemName" runat="server"></telerik:RadDropDownList> --%>
</EditItemTemplate>
<ItemTemplate>
    <asp:Label ID="lblSystemName" runat="server" Text='<%# Bind("SystemName") %>'></asp:Label>
</ItemTemplate>
</telerik:GridTemplateColumn>
<telerik:GridTemplateColumn HeaderText="FISMA System ID"

In [33]:
with open(ctx.get_tempalte_dir()+'temp.aspx', 'r') as f: 
    t=f.read() 
    for d in ldf: 
        print(t.replace("{col}",d['col']).replace("{icn}",d['icn'])  ) 

<CB:DataField  DBColumnName="AgencyName" ImportColumnName="Agency Name"  runat="server"/> 
<CB:DataField  DBColumnName="SystemName" ImportColumnName="System Name"  runat="server"/> 
<CB:DataField  DBColumnName="FISMASystemID" ImportColumnName="FISMA System ID"  runat="server"/> 
<CB:DataField  DBColumnName="FIPS199SystemCategorization" ImportColumnName="FIPS 199 System Categorization"  runat="server"/> 
<CB:DataField  DBColumnName="HighValueAssetID" ImportColumnName="High Value Asset ID"  runat="server"/> 
<CB:DataField  DBColumnName="CryptoModuleName" ImportColumnName="Cryptographic Module Name"  runat="server"/> 
<CB:DataField  DBColumnName="CryptoAlgorithmUsed" ImportColumnName="Cryptographic Algorithm Used"  runat="server"/> 
<CB:DataField  DBColumnName="ServiceProvidedByCryptoModule" ImportColumnName="Service Provided By Cryptographic Module"  runat="server"/> 
<CB:DataField  DBColumnName="LengthCryptoKeysModules" ImportColumnName="Length of associated cryptographic keys or module

In [ ]:
pl=sql_todf("""
SELECT UPPER(LTRIM(RTRIM(DisplayValue))) [Selections], PK_PickList [ML_Value], PK_PickListType [PLT]
FROM vwPickLists   WHERE PK_PickListType > 410 ORDER BY PK_PickListType DESC
""", connstr)  
s=list(pl.loc[pl['PLT']==427]['Selections'])
print(s) 

In [ ]:
base = r"C:\\dev\\CyberScope\\CyberScopeBranch\\CSwebdev\\code\\CyberScope\\HVA\\2023\\"
pages = {
       '3':'4045','4':'4046','5':'4047' 
} 
src = r'C:\\_som\\tmp\\'
for i in pages.keys():
    exts = ['.aspx','.aspx.designer.vb','.aspx.vb']
    for ext in exts:
        with open(f'{src}2023_HVA_MonitoringAndDetection_1{ext}', 'r', encoding='UTF-8') as fr:
            txt = fr.read()
            txt=txt.replace(f'11',f'{i}{i}')
            txt=txt.replace(f'_1',f'_{i}')
            txt=txt.replace(f'"4043"',f'"{pages[i]}"')
            df, code = generate_code_from_db(ctx, qgroup=pages[i])
            txt=txt.replace(f'<!--controls-->',code)
            #print(txt)
        with open(f'{base}2023_HVA_MonitoringAndDetection_{i}{ext}', 'w', encoding='UTF-8') as fw:
            fw.write(txt)

In [ ]:
ip = issue_provider(ctx)  
ip.driver.quit()
#ip.issues
print(ip.issues.to_string())

In [ ]:
ip.driver.quit()

In [ ]:
print(ip.issues.to_string())

In [ ]:
with open(ctx.get_tempalte_dir()+'frmval.sql', 'r') as f:
    s=f.read() 
df=sql_todf("""
SELECT DISTINCT IdText AS [{idt}], REPLACE(IdText, '.','_') [{var}], SortPos  FROM vwQuestions  
WHERE PK_FORM = '2023-Q1-CIO'  AND IdText LIKE '2.%' AND Q_TypeID=2 
ORDER BY SortPos;
 
""", connstr) 
df['sql']=[s.replace('{idt}',x).replace('{var}',y) for x,y in zip(df['{idt}'], df['{var}'])]
 
with open(ctx.get_dest()+'script.sql', 'w') as f:
    f.write( '\n'.join(list(df['sql'])) ) 

In [ ]:
df,code=generate_code_from_db(ctx,4019) 
#generate_section(4018, code)
df[:10]
#sql=df_to_sql(df,ctx,template_name='fsma_QuestionsInsert.sql')

In [ ]:

def parse_grid(tnum, pk, QCol='Question', PivotCols=4): 
    ip = issue_provider(ctx)
    driver=ip.driver
    driver.get(f"https://dayman.cyber-balance.com/jira/si/jira.issueviews:issue-html/CS-{tnum}/CS-{tnum}.html")   
    ldf = pd.read_html(driver.page_source, match="\d\.\d\.\d", header=1)
    df=ldf[0].iloc[:,:].rename(columns={QCol: "QuestionText"})
 
    df['identifier_text'] = df['QuestionText'].apply(lambda s: re.findall('^[\d\.\w]{1,7}',s)[0])
    stypes = {k: chr(v+97) for v, k in enumerate(list(df.columns[1:4]))} 
    print(stypes)
    df=df.loc[:,['QuestionText','identifier_text']] 
    df['CTRLCODE']='LABEL'
    lod=df.to_dict(orient='records')
    l=[]
    for i,r in enumerate(lod):
        l.append(lod[i])
        for k in stypes.keys():
            id=lod[i]['identifier_text']+'.'+stypes[k]
            l.append({
                'identifier_text':id
                , 'QuestionText': id+' '+k
                , 'CTRLCODE':'CNT'
                })
    df=pd.DataFrame(l)
    df['{PK_Question}'] = range(pk,len(df)+pk)
    df['sortpos'] = range(1,len(df)+1) 
    df['FK_QuestionType'] = df['CTRLCODE'].apply(lambda s: 2 if s=='CNT' else 18)
    df['QuestionText']=df['QuestionText'].apply(lambda s: re.sub('^[\d\.\w]{1,7}','',s, count=1).strip())
    df['{QuestionText}']=df['QuestionText']
    df['{idt}'] = df['identifier_text'].str.replace('.','_')
    with open (ctx.get_tempalte_dir()+'fsma_QuestionsInsert.sql', 'r') as f:
        df['sql']=f.read() 
    for i,r in df.iterrows():
        for c in df.columns: 
            df.loc[i,'sql']= re.sub(str(c),str(df.loc[i,c]),df.loc[i,'sql'])  
    sql='\n,'.join(df['sql'])
    with open(f'{ctx.get_dest()}script.sql', 'w') as f:
        f.write(sql)

    gen=script_generator(ctx)
    code=gen.generate(df)
    with open(f'{ctx.get_dest()}script.aspx', 'w') as f:
        f.write(code)
    
    return df,sql,code

# df,sql,code=parse_grid('9295',37400, QCol='Type of Test')
# print(sql)

In [ ]:
ctrl=''
with open(f'{ctx.get_tempalte_dir()}TXT_OPT.aspx', 'r') as f:
    ctrl=f.read().replace('\n','')

def met_provider(s):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)  
    return m[0] if len(m) > 0 else ''

def cell_provider(s, i):  
    if type(s) != str : return s
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s)
    d={0:'a.i',1:'a.ii',2:'b.i',3:'b.ii',4:'c.i',5:'c.ii' } 
    idt=f'{m[0]}.{d[i]}'
    c=ctrl.replace('{idt}',idt.replace('.','_'))
    return f'{c}' if len(m) > 0 else ''

def lbl_provider(s): 
    if type(s) != str : return s
    lbl='<uc:CBlabel ID="CBlabel_{idt}_i" PK_key="{idt}" runat="server" LabelType="Question" />'   
    m=re.findall('\d{1,2}\.\d{1,2}\.?\d{0,2}\.?[\.\d\w]?', s) 
    idt=f'{m[0]}'
    c=lbl.replace('{idt}',idt.replace('.','_'))
    return c
def get_ids():
    dff=sql_todf("SELECT replace(idtext, '.','_') id, pk_question  FROM vwQuestions WHERE PK_QuestionGroup=4014", ctx.config['connstr'])
    print(dff.iloc[0])
    ids = {k:v for k, v in zip(dff['id'], dff['pk_question'] )  }
    return ids
ids=get_ids()   
list(ids.keys())[3:5] 
 

In [ ]:
df=pd.read_excel(r'C:\Users\Tim\Documents\CIO\CIO_SECTION_2.xlsx',  sheet_name='All', header=0 ) # Enterprise Identities  # Public Identities 

#df.fillna('', inplace=True)
#df.drop([0], axis=0, inplace=True)
df.rename(columns={'Unnamed: 2':'.','Unnamed: 4':'..','Unnamed: 6':'...'}, inplace=True)
for i,c in enumerate(df.columns[1:]): 
    df.loc[1:,c] = df.loc[1:, 'Question'].apply(lambda s:  cell_provider(s,i)  )

#mets = [str(m)  for m in list(dff['m']) if '2' in str(m) ]  

df['Question']=df['Question'].apply(lbl_provider )
df.to_html(f'{ctx.get_dest()}parsed.html',index=False)
 
s=''
with open(f'{ctx.get_dest()}parsed.html', 'r') as f:
    s=f.read()
s=re.sub('<th>\.{1,3}</th>','',s)
s=re.sub('th>','td>',s) 
s=re.sub('<td>Number','<td colspan="2">Number',s)

s=re.sub('<thead>|</thead>|<tbody>|</tbody>','',s) 
for k in ids:
    s=s.replace(f'PK_key="{k}"',f'PK_key="{str(ids[k])}"' )
    s=s.replace(f'PK_Question="{k}"',f'PK_Question="{str(ids[k])}"' )
with open(f'{ctx.get_dest()}script.html', 'w') as f: 
    s=html.unescape(s) 
    f.write(s)
df[:5]